In [42]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Load the train and test data
train_df = pd.read_csv("../data/kaggle/train_numeric.csv")
test_df  = pd.read_csv("../data/kaggle/test_numeric.csv")

In [43]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Group                     688 non-null    int64  
 1   Sex                       688 non-null    int64  
 2   Age                       688 non-null    int64  
 3   Patients number per hour  688 non-null    int64  
 4   Arrival mode              688 non-null    int64  
 5   Injury                    688 non-null    int64  
 6   Chief_complain            688 non-null    object 
 7   Mental                    688 non-null    int64  
 8   Pain                      688 non-null    int64  
 9   NRS_pain                  370 non-null    float64
 10  SBP                       670 non-null    float64
 11  DBP                       670 non-null    float64
 12  HR                        672 non-null    float64
 13  RR                        675 non-null    float64
 14  BT        

### Logistic Regression & XGBoost

In [44]:
train_df = train_df.drop(columns=['Length of stay_min','Group','Chief_complain','Diagnosis in ED','KTAS_RN'])
test_df = test_df.drop(columns=['Length of stay_min','Group','Chief_complain','Diagnosis in ED','KTAS_RN'])

In [45]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 688 entries, 0 to 687
Data columns (total 14 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sex                       688 non-null    int64  
 1   Age                       688 non-null    int64  
 2   Patients number per hour  688 non-null    int64  
 3   Arrival mode              688 non-null    int64  
 4   Injury                    688 non-null    int64  
 5   Mental                    688 non-null    int64  
 6   Pain                      688 non-null    int64  
 7   NRS_pain                  370 non-null    float64
 8   SBP                       670 non-null    float64
 9   DBP                       670 non-null    float64
 10  HR                        672 non-null    float64
 11  RR                        675 non-null    float64
 12  BT                        674 non-null    float64
 13  KTAS_expert               688 non-null    int64  
dtypes: float64

In [ ]:
# List of vital signs columns
vital_signs_cols = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp', 'pain']

# Ensure vital signs are numeric and handle missing values
data[vital_signs_cols] = data[vital_signs_cols].apply(pd.to_numeric, errors='coerce')

# Normalize vital signs using Min-Max scaling
scaler = MinMaxScaler()
vital_signs_normalized = scaler.fit_transform(data[vital_signs_cols])

In [46]:
# Fill all NAs with the mean

numeric_cols = train_df.select_dtypes(include=["number"]).columns
for col in numeric_cols:
    train_df[col].fillna(train_df[col].mean(), inplace=True)
    test_df[col].fillna(test_df[col].mean(), inplace=True)


/tmp/ipykernel_132386/1146406248.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].mean(), inplace=True)
/tmp/ipykernel_132386/1146406248.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [48]:
# Optionally, drop rows with missing target values (if any)
train_df = train_df.dropna(subset=["KTAS_expert",])
test_df  = test_df.dropna(subset=["KTAS_expert", ])

# Define the target column and feature set.
# Here we assume that all columns except 'KTAS_expert' are features.
target_col = "KTAS_expert"
features = [col for col in train_df.columns if col != target_col]

X_train = train_df[features]
y_train = train_df[target_col]

X_test = test_df[features]
y_test = test_df[target_col]

# Build and train the Logistic Regression model.
# For multiclass problems, LogisticRegression can use multinomial mode.
logreg = LogisticRegression(max_iter=1000, multi_class='auto')
logreg.fit(X_train, y_train)

# Make predictions on the test set.
preds_lr = logreg.predict(X_test)

# Print the results.
print("=== Logistic Regression Predictions ===")
print("\nAccuracy:", accuracy_score(y_test, preds_lr))
print("\nClassification Report:\n", classification_report(y_test, preds_lr))


=== Logistic Regression Predictions ===

Accuracy: 0.5146804835924007

Classification Report:
               precision    recall  f1-score   support

           1       0.73      0.67      0.70        12
           2       0.59      0.09      0.15       148
           3       0.55      0.66      0.60       217
           4       0.49      0.73      0.59       183
           5       0.06      0.05      0.05        19

    accuracy                           0.51       579
   macro avg       0.48      0.44      0.42       579
weighted avg       0.53      0.51      0.47       579



/home/josephL/miniconda3/envs/llm-bias/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/josephL/miniconda3/envs/llm-bias/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [49]:
y_train = y_train - 1

In [50]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb

# Build and train the XGBoost model.
# Setting 'use_label_encoder' to False to avoid warnings and specifying an evaluation metric.
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
xgb_model.fit(X_train, y_train)

# Make predictions on the test set.
preds_xgb = xgb_model.predict(X_test)

# Print the results.
print("=== XGBoost Predictions ===")
print("\nAccuracy:", accuracy_score(y_test, preds_xgb))
print("\nClassification Report:\n", classification_report(y_test, preds_xgb))


=== XGBoost Predictions ===

Accuracy: 0.30569948186528495

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.03      0.08      0.04        12
           2       0.26      0.20      0.22       148
           3       0.34      0.64      0.44       217
           4       0.53      0.04      0.08       183
           5       0.00      0.00      0.00        19

    accuracy                           0.31       579
   macro avg       0.19      0.16      0.13       579
weighted avg       0.36      0.31      0.25       579



/home/josephL/miniconda3/envs/llm-bias/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [20:26:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/home/josephL/miniconda3/envs/llm-bias/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/josephL/miniconda3/envs/llm-bias/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/josephL/miniconda3/envs/llm-bias/lib/python3.12/site-packages/sklearn/me

### BioBERT + MLP

In [26]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [140]:
train_df = pd.read_csv("../data/kaggle/train_numeric.csv")
test_df  = pd.read_csv("../data/kaggle/test_numeric.csv")
train_df = train_df.drop(columns=['Group','KTAS_RN'])
test_df = test_df.drop(columns=['Group','KTAS_RN'])

# Optionally, drop rows with missing target values (if any)
train_df = train_df.dropna(subset=["Diagnosis in ED","Chief_complain"])
test_df  = test_df.dropna(subset=["Diagnosis in ED","Chief_complain" ])

In [138]:

# Load the pre-trained SentenceTransformer model for clinical text
# This model is fine-tuned for biomedical and clinical text embeddings
model_name = 'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb'
model = SentenceTransformer(model_name)

# from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
# model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


In [141]:
# Process chief complaints in batches of 10,000
print("Computing symptom embeddings in batches...")
# Create embeddings for the chiefcomplaint column for train and test datasets
train_texts = train_df['Chief_complain'].tolist()
test_texts = test_df['Chief_complain'].tolist()

# Use the model to encode the texts; show_progress_bar=True gives you a progress update
train_embeddings = model.encode(train_texts, show_progress_bar=True)
test_embeddings = model.encode(test_texts, show_progress_bar=True)

train_texts = train_df['Diagnosis in ED'].tolist()
test_texts = test_df['Diagnosis in ED'].tolist()

# Use the model to encode the texts; show_progress_bar=True gives you a progress update
train_embeddings_two = model.encode(train_texts, show_progress_bar=True)
test_embeddings_two = model.encode(test_texts, show_progress_bar=True)


Computing symptom embeddings in batches...


Batches: 100%|██████████| 19/19 [00:00<00:00, 127.80it/s]


In [142]:
test_embeddings.shape

(577, 768)

In [122]:
# Save the embeddings as .npy files using NumPy
np.save('../data/kaggle/KTAS_train_chiefcomplaint_embeddings.npy', train_embeddings)
np.save('../data/kaggle/KTAS_test_chiefcomplaint_embeddings.npy', test_embeddings)

In [133]:
import numpy as np
from sklearn.decomposition import PCA

# ----- For the Training Set -----

# Convert train symptom embeddings to a numpy array
train_embeddings = np.array(train_embeddings)

# Perform PCA on train symptom embeddings to reduce dimensionality to 10
print("Performing PCA on train symptom embeddings...")
pca = PCA(n_components=25)
train_embeddings = pca.fit_transform(train_embeddings)

# Save the reduced and comprehensive train embeddings to disk
# np.save('../data/mimic-iv-private/train_symptom_embeddings_reduced.npy', train_symptom_embeddings_reduced)


# ----- For the Test Set -----

# Convert test symptom embeddings to a numpy array
test_embeddings = np.array(test_embeddings)

# Use the same PCA transformation fitted on the training set to transform the test symptom embeddings
print("Performing PCA on test symptom embeddings...")
test_embeddings = pca.transform(test_embeddings)

# Save the reduced and comprehensive test embeddings to disk
# np.save('../data/mimic-iv-private/test_symptom_embeddings_reduced.npy', test_embeddings)


Performing PCA on train symptom embeddings...
Performing PCA on test symptom embeddings...


In [143]:
# Fill all NAs with the mean

numeric_cols = train_df.select_dtypes(include=["number"]).columns
for col in numeric_cols:
    train_df[col].fillna(train_df[col].mean(), inplace=True)
    test_df[col].fillna(test_df[col].mean(), inplace=True)


/tmp/ipykernel_132386/1146406248.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(train_df[col].mean(), inplace=True)
/tmp/ipykernel_132386/1146406248.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=

In [144]:
train_df = train_df.drop(columns=['Chief_complain','Diagnosis in ED'])
test_df = test_df.drop(columns=['Chief_complain','Diagnosis in ED'])

# Define the target column and feature set.
# Here we assume that all columns except 'KTAS_expert' are features.
target_col = "KTAS_expert"
features = [col for col in train_df.columns if col != target_col]

X_train = train_df[features]
y_train = train_df[target_col].values

X_test = test_df[features]
y_test = test_df[target_col].values

# # (Optional) Convert all features to numeric in case they are not.
# X_train = X_train.apply(pd.to_numeric, errors='coerce')
# X_test  = X_test.apply(pd.to_numeric, errors='coerce')

# # (Optional) Fill any remaining missing values with the median of each column.
# X_train.fillna(X_train.median(), inplace=True)
# X_test.fillna(X_test.median(), inplace=True)


In [145]:
X_train = np.hstack([X_train.values, train_embeddings, train_embeddings_two])  # Combine vitals and embeddings
X_test = np.hstack([X_test.values, test_embeddings, test_embeddings_two])  # Replace with inference embeddings if different

In [146]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import os 
import sys
sys.path.append(os.path.abspath('..'))
import utils.utils as utils 

metrics_results = []

# Train an MLP classifier
model = MLPClassifier(
    hidden_layer_sizes=(1000, 1000),  # Small network
    random_state=42,
    early_stopping=True
)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the predictions
evaluation_metrics = utils.evaluate_predictions(y_pred,y_test,ordinal=True, by_class=True)

# Store results
metrics_results.append(evaluation_metrics)

# Display results
print(metrics_results[0])

{'overall': {'accuracy': 0.44540727902946275, 'precision': 0.5118035166261915, 'recall': 0.44540727902946275, 'f1_score': 0.38962643268049574, 'adjusted_accuracy': 0.949740034662045, 'adjusted_precision': 0.949071457079619, 'adjusted_recall': 0.949740034662045, 'adjusted_f1': 0.9459780814305845, 'mae': 0.610051993067591, 'mse': 0.7348353552859619, 'quadratic_kappa': np.float64(0.3550863318834848)}, 'by_class': {'1': {'precision': 0.375, 'recall': 0.25, 'f1-score': 0.3, 'support': 12.0}, '2': {'precision': 0.4909090909090909, 'recall': 0.1836734693877551, 'f1-score': 0.26732673267326734, 'support': 147.0}, '3': {'precision': 0.42247191011235957, 'recall': 0.8663594470046083, 'f1-score': 0.56797583081571, 'support': 217.0}, '4': {'precision': 0.6730769230769231, 'recall': 0.19230769230769232, 'f1-score': 0.29914529914529914, 'support': 182.0}, '5': {'precision': 0.23529411764705882, 'recall': 0.21052631578947367, 'f1-score': 0.2222222222222222, 'support': 19.0}, 'accuracy': 0.44540727902